# Assignment 5 (Hands-on: Convolutional Neural Networks for Image Segementation) - (15 Pts)

In this homework, we will explore segmentation and data preprocessing with convolutional neural networks.

## Conda Setup

Generate the conda environment by running in the terminal:

`conda create --name gcv_exercise_5 python=3.8`

Run to activate environment:

`conda activate gcv_exercise_5`

Install all required packages for this exercise by running:

`conda install matplotlib numpy scikit-image scikit-learn tqdm pandas`

Install PyTorch and its prerequisites:

`conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia`

Install Albumentations, pandas, seaborn and ipykernel for the preprocessing

`conda install -c conda-forge albumentations pandas ipykernel seaborn`

(Optional) If you are working on the TUWEL Juypter Notebooks:

`ipython kernel install --name "GCV_E_5" --user`

## Notebook Setup & Imports

In [ ]:
# Import necessary libraries

import math
import random
from random import randint
from tqdm import trange
import pandas as pd
import numpy as np
import albumentations as A # Scikit-Image and -Learn library for image augmentation
from albumentations.pytorch import ToTensorV2
from PIL import Image
import matplotlib.pyplot as plt # useful plotting library for python
import seaborn as sns # useful plotting library for python
import torch # framework for deep learning
import torchvision
import torch.nn as nn
import torchvision.models as models

# This code is to make matplotlib figures appear inline in the notebook rather than in a new window.

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Set random seeds for reproducibility

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

# Define constants for masks

MASK_FG = 0
MASK_BG = 1
MASK_IGNORE = 2

## Part 1: Data preparation

When it comes to deep learning, the quality and quantity of our data can make or break the performance of our model. Therefore, in this section, we'll be preparing our dataset in a format that is most suitable for use in the remainder of our framework.

For image-based tasks, two commonly used packages for data preparation and transformation are torchvision and albumentations. These packages provide a range of data loaders for common datasets and various image transformation operations that we can use to preprocess our data.

Specifically, we'll be working with the oxford-iiit-pet dataset in this section. Our aim is to apply a series of data augmentations to the images to increase the diversity and quantity of our training data, while also normalizing the images to facilitate more effective model training.

To perform these augmentations, we'll utilize the functions provided by the torchvision and albumentations packages. The input images can be found in the data/oxford-iiit-pet/images/ directory, while the corresponding masks are located in the data/oxford-iiit-pet/annotations/trimaps/ directory. Note that all images should be resized to 64x64 after the augmentation step. The dataset will be automatically downloaded by the next cell.

In [ ]:
torchvision.datasets.OxfordIIITPet("data", download=True)

df = pd.read_csv(
    "data/oxford-iiit-pet/annotations/list.txt",
    delimiter=" ",
    skiprows=6,
    header=None,
    names=["stem", "class_id", "species", "breed"]
)
df["class_name"] = df.stem.map(lambda x: x.split("_")[0])
df["image"] = df.stem.map(lambda x: f"data/oxford-iiit-pet/images/{x}.jpg")
df["trimap"] = df.stem.map(lambda x: f"data/oxford-iiit-pet/annotations/trimaps/{x}.png")

df

### Visualize dataset information

Before diving into the model training phase, it's essential to gain a high-level overview of the data we're working with. This can be achieved using visualization tools such as Matplotlib and Seaborn. Seaborn, in particular, is a popular Python data visualization library that offers a user-friendly interface for creating informative and aesthetically pleasing statistical graphics.

By visualizing the data in this way, we can identify patterns and trends in the input images and labels, which can help us make informed decisions about our model design and training strategy. The resulting plot should resemble the following visualization:

<center><img src="imgs/example_data_visualization.png" alt="Plot" width="960" /></center>

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15,5))
# *****BEGINNING OF YOUR CODE (DO NOT DELETE THIS LINE)*****
# the countplot to make the rectangles, ax, to specify that they are in the same plot
sns.countplot(x='class_id', data=df, ax=axs[0])
sns.countplot(x='species', data=df, ax=axs[1])
sns.countplot(x='breed', data=df, ax=axs[2])
plt.show()

# *****END OF YOUR CODE (DO NOT DELETE THIS LINE)*****

### Visualize dataset example

Now, let's take a moment to visualize a sample from our training dataset. To start, we'll focus on the first image in the dataset, which should give us a good idea of what the training data looks like.

By examining the training data in more detail, we can better understand the characteristics of the input images and the corresponding labels, which can inform our model design and hyperparameter selection. So, without further ado, let's take a look at the first image in the dataset. It should resemble the following visualization:

<center><img src="imgs/example_image.png" alt="Plot" width="960" /></center>

In [ ]:
print(df.iloc[0])
# *****BEGINNING OF YOUR CODE (DO NOT DELETE THIS LINE)*****

image1 = Image.open(df.iloc[0]['image'])
image2 = Image.open(df.iloc[0]['trimap'])
fig = plt.figure(figsize=(10,5))

# we had to deactivate the axis
fig.add_subplot(1, 2, 1)
plt.imshow(image1)
plt.axis('off')

fig.add_subplot(1, 2, 2)
plt.imshow(image2)
plt.axis('off')

plt.show()
# *****END OF YOUR CODE (DO NOT DELETE THIS LINE)*****

### Implement the dataloader

Moving forward, we'll be creating a dataloader to streamline the process of loading and augmenting our data. To do this, we'll need to implement the __getitem__ function and define the specific image augmentation techniques we'll be using. Please note, the images may not all be RGB, so you have to convert them to RGB (.convert('RGB')). 

The augmentations we'll be applying during training are:
- Flipping the image horizontally and vertically
- Randomly scaling the image
- Randomly rotating the image
- Random brightness and contrast
- Resizing the image to 64 x 64
- Normalization with the imagenet mean and std

To ensure a fair and unbiased evaluation, it is important to keep the validation set representative of the original data distribution without introducing any artificial modifications. This allows us to accurately assess the model's performance on unseen data and make reliable comparisons between different models or approaches. There for we will only apply the resizing and the normalization to the imagenet mean and std. The output of the augmented image should be a pytorch tensor.

By automating the image augmentation process, we can efficiently generate a diverse range of training samples, which in turn helps to improve the robustness and generalization of our model. So, let's get started by defining the __getitem__ function and specifying the desired data augmentation methods.

Note: The trimap has three pixel annotations: 1: Foreground, 2: Background, 3: Not classified. For training you have to adapt these, such that they start with 0. (0: Foreground, 1: Background, 2: Not classified)

Note: the `nn.CrossEntropyLoss()` loss expects the mask to be of type long.

In [ ]:
class IIITDataset(torch.utils.data.Dataset):
    """IIIT dataset.
    Args:
        df (pandas.DataFrame): Pandas dataframe obtained from the Oxford-IIIT Pet Dataset.
        tfm (albumentations.Compose): composed albumentations transforms.

    Methods:
        __len__: returns the length of the dataset.
        __getitem__: returns a tuple of (image, mask) where mask is a 2D numpy array.
    """
    def __init__(self, df, tfm=None):
        """Initialize the dataset.

        Args:
            df (pandas.DataFrame): Pandas dataframe obtained from the Oxford-IIIT Pet Dataset.
            tfm (albumentations.Compose): composed albumentations transforms.
        """
        self.df = df
        self.tfm = tfm
    def __len__(self):
        """Return the length of the dataset."""
        return len(self.df)
    def __getitem__(self, i):
        """Return a tuple of (image, mask) where mask is a 2D numpy array.
        
        Args:
            i (int): index of the image in the dataset.

        Returns:
            tuple: (image, mask) where mask is a 2D numpy array.
        """
        # *****BEGINNING OF YOUR CODE (DO NOT DELETE THIS LINE)*****
        
        image = Image.open(self.df.iloc[i]['image']).convert('RGB')
        mask = Image.open(self.df.iloc[i]['trimap'])

        # as it needs to start from 0, we decrement by 1
        image = np.array(image)
        mask = np.array(mask) - 1 
        
        # apply the augmentations
        augmentation = self.tfm(image=image, mask=mask)

        tens = ToTensorV2()(image=augmentation['image'],mask = augmentation['mask'])
        # it is a dictionary containing the image
        image = tens['image']

        mask = tens['mask']
        # we need a long type
        mask = mask.long()
        # and we also need to return a numpy
        mask = mask.numpy()
        return image,mask
        # *****END OF YOUR CODE (DO NOT DELETE THIS LINE)*****

imagenet_mean = (0.485, 0.456, 0.406)
imagenet_std = (0.229, 0.224, 0.225)

# transformations for the train and validation sets
train_tfm = A.Compose([
    # *****BEGINNING OF YOUR CODE (DO NOT DELETE THIS LINE)*****
    A.RandomRotate90(),
    A.VerticalFlip(),
    A.HorizontalFlip(),
    A.RandomScale(),
    A.RandomBrightness(),
    A.RandomBrightnessContrast(),
    A.Resize(height=64,width=64),
    A.Normalize(mean=imagenet_mean, std=imagenet_std)
    # *****END OF YOUR CODE (DO NOT DELETE THIS LINE)*****
])
# Only normalize, resize and convert to tensor for validation
# to keep the validation deterministic
val_tfm = A.Compose([
    # *****BEGINNING OF YOUR CODE (DO NOT DELETE THIS LINE)*****
    A.Resize(64, 64),
    A.Normalize(mean=imagenet_mean, std=imagenet_std),
    ToTensorV2()
    # *****END OF YOUR CODE (DO NOT DELETE THIS LINE)*****
])

def imagenet_denorm(x):
    """x: array-like with shape (..., H, W, C)"""
    return x * imagenet_std + imagenet_mean

dataset = IIITDataset(df, tfm=train_tfm)

print(len(dataset))

# split dataset into train and test
train_set, validation_set = torch.utils.data.random_split(dataset, [len(dataset) - 2000, 2000])

# create dataloaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=2048, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=2048, shuffle=False)

### Visualize augmented images with masks

Before proceeding further, it's important to conduct a sanity check to verify the integrity of our data. One approach we can take is to visually inspect a sample of images and their corresponding masks from the augmented dataset. This allows us to ensure that our data augmentation process is functioning correctly and not inadvertently introducing any errors or biases into the training data.

To give you an idea, here's an example of what this might look like:

<center><img src="imgs/example_aug.png" alt="Plot" width="960" /></center>

Note: Matplotlib will automatically clip the image and the mask. Normalize them between 0 and 1 before plotting them. Warning: 

`Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).`

In [ ]:
# visualize some images
for _ in range(3):
    # *****BEGINNING OF YOUR CODE (DO NOT DELETE THIS LINE)*****

    df_length = len(df)
    img, mask = IIITDataset(df=df, tfm=train_tfm).__getitem__(random.randint(0, df_length))

    # as it has 3 channels
    img = img.permute(1, 2, 0).numpy()
    img = img - np.min(img)
    norm_img = img / np.max(img)
    mask = mask - np.min(mask)
    norm_mask = mask / np.max(mask)

    # just showing the figure
    fig = plt.figure(figsize=(10,5))

    fig.add_subplot(1, 2, 1)
    plt.imshow(norm_img)
    plt.axis('off')

    fig.add_subplot(1, 2, 2)
    plt.imshow(norm_mask, cmap='gray')
    plt.axis('off')

    plt.show()
    
    # *****END OF YOUR CODE (DO NOT DELETE THIS LINE)*****



## Part 2: Implementing the network

### Model Definition

We use a pretrained MobileNetV2 as our encoder and then implement our own decoder. We return three different layers for the skip connections of our "U-Net". You will implement the U-Net-Decoder and then train it on the dataset. An example overview of a complete U-Net architecture is presented here:

<center><img src="imgs/unet.png" alt="Plot" width="960" /></center>

However, our U-Net will be simpler, to reduce training time, and will only have 3 down- and upsampling steps and therefore, only two skip connections (the last upsampling is done via bilinear interpolation, to futher speed up training). For the implementation you will need these torch.nn functions:

- nn.Conv2d          (https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)
- nn.BatchNorm2d     (https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html)
- nn.ReLU6           (https://pytorch.org/docs/stable/generated/torch.nn.ReLU6.html)
- nn.ConvTranspose2d (https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html)

ReLU6 is a modification of the `rectified linear unit` where we limit the activation to a maximum size of 6.

The final architecture should look similar to this:

- `nn.ConvTranspose2d`(kernel_size=(4,4), stride=2, padding=1, bias=True)
- `InvertedResidual`(stride=1, expand_ratio=6)
- `nn.ConvTranspose2d`(kernel_size=(4,4), stride=2, padding=1, bias=True)
- `InvertedResidual`(stride=1, expand_ratio=6)
- `nn.Conv2d`(kernel_size=(1,1))
- `nn.Conv2d`(kernel_size=(1,1))
- `torch.sigmoid`

where InvertedResidual is composed of:

- `nn.Conv2d`(kernel_size=(1,1), stride=1, padding=0, bias=False)
- `nn.BatchNorm2d`
- `nn.ReLU6`
- `nn.Conv2d`(kernel_size=(3,3), stride=stride, padding=1, bias=False)
- `nn.BatchNorm2d`
- `nn.ReLU6`
- `nn.Conv2d`(kernel_size=(1,1), stride=1, padding=0, bias=False)
- `nn.BatchNorm2d`

The dimensions of the layers can be seen here:

| Layer (type)               | Output Shape (B, C, H, W) |
|----------------------------|---------------------------|
| ├─MobileNetV2_Encoder: 1-1 | [64, 32, 8, 8]
| ├─UNet_Decoder: 1-2        | [64, 3, 32, 32]           |
| │  └─ConvTranspose2d: 2-2  | [64, 24, 8, 8]            |
| │  └─InvertedResidual: 2-3 | [64, 24, 8, 8]            |
| │  └─ConvTranspose2d: 2-4  | [64, 16, 16, 16]          |
| │  └─InvertedResidual: 2-5 | [64, 16, 16, 16]          |
| │  └─Conv2d: 2-6           | [64, 16, 32, 32]          |
| │  └─Conv2d: 2-7           | [64, 3, 32, 32]           |

where B is the batchsize, C is the number of channels, and H and W are the height and width, respectively. The hidden dimensions within the InvertedResidual layer are defined by `hidden_dim = round(inp * expand_ratio)`. 

In [ ]:
class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()

        hidden_dim = round(inp * expand_ratio)

        # *****BEGINNING OF YOUR CODE (DO NOT DELETE THIS LINE)*****

        layers = []
        layers.extend([
            nn.Conv2d(inp, hidden_dim, kernel_size=(1,1), stride=1, padding=0, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=(3,3), stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),
            nn.Conv2d(hidden_dim, oup, kernel_size=(1,1), stride=1, padding=0, bias=False),
            nn.BatchNorm2d(oup),
        ])

        self.conv = nn.Sequential(*layers)
        # *****END OF YOUR CODE (DO NOT DELETE THIS LINE)*****

    def forward(self, x):

        return self.conv(x)

class MobileNetV2_Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.mobilenet = models.mobilenet_v2(weights="MobileNet_V2_Weights.IMAGENET1K_V1")

        for param in self.mobilenet.parameters():
            param.requires_grad = False
        
    def forward(self, x):
        for n in range(0, 2):
            x = self.mobilenet.features[n](x)
        x1 = x

        for n in range(2, 4):
            x = self.mobilenet.features[n](x)
        x2 = x

        for n in range(4, 7):
            x = self.mobilenet.features[n](x)
        x3 = x

        return x1, x2, x3

# UNet Decoder
class UNet_Decoder(torch.nn.Module):
    def __init__(self, n_classes=1):
        super().__init__()

        # *****BEGINNING OF YOUR CODE (DO NOT DELETE THIS LINE)*****
        self.dconv1 = nn.ConvTranspose2d(32, 24, kernel_size=(4,4), stride=2, padding=1, bias=True)
        self.invres1 = InvertedResidual(48, 24, stride=1, expand_ratio=6)

        self.dconv2 = nn.ConvTranspose2d(24, 16, kernel_size=(4,4), stride=2, padding=1, bias=True)
        self.invres2 = InvertedResidual(32, 16, stride=1, expand_ratio=6)

        self.conv_last = nn.Conv2d(16, 16, kernel_size=(1,1))
        self.conv_score = nn.Conv2d(16, 3, kernel_size=(1,1))

        self.sigmoid = torch.sigmoid

        # *****END OF YOUR CODE (DO NOT DELETE THIS LINE)*****

    def forward(self, x):
        x1, x2, x3 = x
        # HINT:
        # print(x1.shape, x2.shape, x3.shape)
        # torch.Size([B, 16, 32, 32]) torch.Size([B, 24, 16, 16]) torch.Size([B, 32, 8, 8])
        # x2.shape == self.dconv1(x3)
        up3 = torch.cat([
            x2,
            self.dconv1(x3)
        ], dim=1)
        up3 = self.invres1(up3)

        # x1.shape == self.dconv2(up3)
        up4 = torch.cat([
            x1,
            self.dconv2(up3)
        ], dim=1)
        up4 = self.invres2(up4)

        up5 = torch.nn.functional.interpolate(up4, scale_factor=2, mode='bilinear', align_corners=False)

        x = self.conv_last(up5)

        x = self.conv_score(x)


        x = self.sigmoid(x)

        return x


### Training Loop

You shouldn't need to change anything here, except for maybe the number of epochs. An example output plot is shown here:

<center><img src="imgs/loss_and_accuracy.png" alt="Plot" width="960" /></center>

The network was trained for 50 epochs. However, this might take a while to train depending on your hardware. Though, as you can see, after 10 epochs we are already at around 75% accuracy. Which should be sufficient for early experiments, so you can quickly see wether you approach works.

In [ ]:
# Training
def train(model, loader, optimizer, loss_fn, device, epoch):
    model.train()
    train_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    return train_loss / len(loader)

def validate(model, loader, loss_fn, device, epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x).squeeze(1)
            loss = loss_fn(pred, y)
            test_loss += loss.item()

            _, pred = torch.max(pred, 1)
            correct = torch.mean((pred == y).type(torch.float64))
            print("-> Epoch: {:.1f}. Validation.  Accuracy: {:.3f}".format(epoch, correct.cpu().numpy()))
    return test_loss / len(loader), correct.cpu().numpy()

def train_loop(model, train_loader, validation_loader, loss_fn, optimizer, device, epochs=10):
    train_losses, test_losses, test_accuracy = [], [], []
    for epoch in trange(epochs):
        train_loss = train(model, train_loader, optimizer, loss_fn, device, epoch)
        test_loss, accuracy = validate(model, validation_loader, loss_fn, device, epoch)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        test_accuracy.append(accuracy)
    return train_losses, test_losses, test_accuracy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

encoder = MobileNetV2_Encoder().to(device)
decoder = UNet_Decoder().to(device)
model = torch.nn.Sequential(encoder, decoder).to(device)
model.to(device)

# from torchinfo import summary
# print(summary(encoder.mobilenet, input_size=(64, 3, 32, 32)))
# print(summary(model, input_size=(64, 3, 32, 32)))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses, test_losses, test_accuracy = train_loop(model, train_loader, validation_loader, criterion, optimizer, device, epochs=2)

plt.subplot(1,2,1)
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.subplot(1,2,2)
plt.plot(test_accuracy, label="test accuracy")
plt.legend()
plt.show()

## Part 3: Evaluation

In order to gain a deeper understanding of our network's performance, we will be analyzing a selection of its predictions through both qualitative and quantitative methods. Let's begin by focusing on the qualitative analysis, where we'll take a closer look at some visualizations of the predictions.

For context, qualitative analysis involves evaluating the subjective aspects of data, such as visual appearance, whereas quantitative analysis involves examining objective data, such as numerical measurements or statistical analysis. By combining both approaches, we can gain a more comprehensive understanding of our model's strengths and limitations.

### Qualitative Analysis

The network should predict masks similiar in quality to this (However, some images are harder to predict and thus result in worse masks):

<center><img src="imgs/network_output.png" alt="Plot" width="960" /></center>

In [ ]:
# Visualize some predictions
model.eval()
with torch.no_grad():
    for x, y in validation_loader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        _, pred = torch.max(pred, 1)
        break

for i in range(3):
    # *****BEGINNING OF YOUR CODE (DO NOT DELETE THIS LINE)*****
    fig = plt.figure(figsize=(12, 6))
    # just put them on the same collumn
    fig.add_subplot(1, 3, 1)
    
    # normalised it to get rid of the warning
    img  = x[i].numpy().transpose((1, 2, 0))
    img = img - np.min(img) 
    img = img / np.max(img)

    plt.imshow(img)
    plt.axis('off')

    # truth
    fig.add_subplot(1, 3, 2)
    plt.imshow(y[i])
    plt.axis('off')

    # prediction
    fig.add_subplot(1, 3, 3)
    plt.imshow(pred[i])
    plt.axis('off')

    plt.show()    
    # *****END OF YOUR CODE (DO NOT DELETE THIS LINE)*****

### Quantitative Analysis

In order to ensure the accuracy of our model on the test set, we will be implementing the Intersection over Union (IoU) metric. It's worth noting that in a real-world scenario, we would typically use a separate test set for this purpose. However, for the purposes of this exercise, we will be utilizing the validation set. A reasonable IoU to achive for this exercise is `0.45`, however, depending on the training time it may also be lower. In this case increasing the epochs might increase the IoU, especially, if the loss of the validation data is still decreasing.

In [ ]:
def iou(pred, target):
    # *****BEGINNING OF YOUR CODE (DO NOT DELETE THIS LINE)*****

    # *****END OF YOUR CODE (DO NOT DELETE THIS LINE)*****

In [ ]:
model.eval()

with torch.no_grad():
    iou_list = []
    for inputs, targets in validation_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)

        preds = torch.argmax(outputs, dim=1)  # get predicted segmentation map

        iou_val = iou(preds, targets)  # compute IoU
        iou_list.append(iou_val)

mean_iou = sum(iou_list) / len(iou_list)
print('Mean IoU:', mean_iou)